In [ ]:
import numpy as np
from scipy.fft import fft2, ifft2
import matplotlib.pyplot as plt

# Von Kármán spectrum function with azimuthal dependence
def von_karman_spectrum(kx, ky, H, kn, ks, D, azimuth):
    """
    Create the von Kármán spectrum based on the abyssal hill parameters.
    kx, ky: Wavenumber arrays
    H: rms height
    kn, ks: Characteristic widths (normal and strike directions)
    D: Fractal dimension
    azimuth: Ridge orientation angle in radians (azimuth of abyssal hill fabric)
    """
    # Rotate the wavenumbers by the azimuth angle
    kx_rot = kx * np.cos(azimuth) + ky * np.sin(azimuth)
    ky_rot = -kx * np.sin(azimuth) + ky * np.cos(azimuth)
    
    # Dimensionless wavenumbers scaled by the characteristic widths
    kx_dim = kx_rot / kn
    ky_dim = ky_rot / ks
    
    # Von Kármán power spectral density
    k_squared = kx_dim**2 + ky_dim**2
    spectrum = (H**2) * (1 + k_squared)**(-(D+1)/2)
    return spectrum

# Random phase for the noise field
def generate_random_phase(nx, ny):
    """ Generate a random field with normally distributed random phases. """
    return np.exp(2j * np.pi * np.random.rand(nx, ny))

# Sediment modification for abyssal hill parameters
def modify_by_sediment(H, kn, ks, sediment_thickness):
    """
    Modify abyssal hill parameters based on sediment thickness.
    Sediment drape reduces the rms height (H) and increases the width (kn, ks).
    """
    H_sed = np.maximum(H - sediment_thickness / 2, 0)  # Modify H based on sediment thickness
    kn_sed = kn + 1.3 * kn * (sediment_thickness / H)  # Modify kn based on sediment
    ks_sed = ks + 1.3 * ks * (sediment_thickness / H)  # Modify ks similarly
    return H_sed, kn_sed, ks_sed

# Azimuth calculation from seafloor age gradient
def calculate_azimuth_from_age(seafloor_age):
    """
    Calculate the azimuth from the gradient of seafloor age.
    Azimuth is perpendicular to the gradient vector.
    """
    # Compute the gradients in x and y direction
    grad_y, grad_x = np.gradient(seafloor_age)
    
    # Compute azimuth as the angle of the gradient vector (radians)
    azimuth = np.arctan2(grad_y, grad_x)
    
    return azimuth

# Generate synthetic bathymetry with variable azimuth and sediment modification
def generate_synthetic_bathymetry(grid_size, seafloor_age, sediment_thickness, params):
    """
    Generate synthetic bathymetry using a von Kármán model with azimuthal orientation
    and sediment thickness modification.
    
    grid_size: tuple (nx, ny) specifying the size of the grid
    seafloor_age: 2D array of seafloor ages (used to calculate azimuth)
    sediment_thickness: 2D array of sediment thicknesses
    params: Dictionary containing the base abyssal hill parameters for each grid
            e.g., {'H': H, 'kn': kn, 'ks': ks, 'D': D}
    """
    nx, ny = grid_size
    
    # Calculate azimuth from seafloor age gradient
    azimuth = calculate_azimuth_from_age(seafloor_age)
    
    # Create wavenumber arrays (kx, ky)
    kx = np.fft.fftfreq(nx) * 2 * np.pi
    ky = np.fft.fftfreq(ny) * 2 * np.pi
    kx, ky = np.meshgrid(kx, ky)
    
    # Modify parameters based on sediment thickness
    H_sed, kn_sed, ks_sed = modify_by_sediment(params['H'], params['kn'], params['ks'], sediment_thickness)
    
    # Generate the von Kármán spectrum with azimuth
    spectrum = von_karman_spectrum(kx, ky, H_sed, kn_sed, ks_sed, params['D'], azimuth)
    
    # Generate random phase field
    random_phase = generate_random_phase(nx, ny)
    
    # Multiply the spectrum by random phase
    bathymetry_fft = np.sqrt(spectrum) * random_phase
    
    # Perform inverse FFT to get the synthetic bathymetry in space domain
    synthetic_bathymetry = np.real(ifft2(bathymetry_fft))
    
    return synthetic_bathymetry

# Example of input parameters (these would be derived from your grids)
params = {
    'H': 50,       # Base rms height in meters
    'kn': 0.2,   # Characteristic width in km^-1 (normal to ridge)
    'ks': 0.02,   # Characteristic length in km^-1 (parallel to ridge)
    'D': 2.2       # Fractal dimension
}

# Example grids (replace with real data)
nx, ny = 512, 512
#seafloor_age = np.random.rand(nx, ny) * 100  # Example seafloor age in Myr
seafloor_age,sediment_thickness = np.meshgrid(np.arange(0,512,1),np.arange(0,512,1))
#sediment_thickness = np.random.rand(nx, ny) * 500  # Example sediment thickness in meters
seafloor_age = seafloor_age+(sediment_thickness*2)
# Generate the synthetic bathymetry
synthetic_bathymetry = generate_synthetic_bathymetry((nx, ny), 
                                                     seafloor_age, 
                                                     sediment_thickness*2, 
                                                     params)

# Optionally, plot the bathymetry
plt.imshow(synthetic_bathymetry, cmap='terrain')
plt.colorbar(label="Depth (m)")
plt.title("Synthetic Abyssal Hill Bathymetry with Sediment and Azimuth")
plt.show()


In [ ]:
import numpy as np
from scipy.ndimage import convolve
from concurrent.futures import ThreadPoolExecutor, as_completed

# Generate a random noise field
def generate_random_field(grid_size):
    """ Generate a random field with normally distributed random values. """
    return np.random.randn(*grid_size)

# Generate a spatial filter based on local parameters
def generate_spatial_filter(H, kn, ks, azimuth, filter_size=25):
    """
    Generate a spatial filter (Gaussian-like) based on local parameters H, kn, ks, and azimuth.
    The filter size controls how large the filter window is.
    
    H: RMS height
    kn, ks: Characteristic widths (normal and parallel to ridge)
    azimuth: Ridge orientation in radians
    """
    # Create a 2D grid for the filter
    x = np.linspace(-1, 1, filter_size)
    y = np.linspace(-1, 1, filter_size)
    xx, yy = np.meshgrid(x, y)
    
    # Rotate the coordinates based on the azimuth
    x_rot = xx * np.cos(azimuth) + yy * np.sin(azimuth)
    y_rot = -xx * np.sin(azimuth) + yy * np.cos(azimuth)
    
    # Apply anisotropic scaling based on kn and ks
    filter_exp = -(x_rot**2 / (2 * kn**2) + y_rot**2 / (2 * ks**2))
    
    # Generate a Gaussian-like filter and scale it by the RMS height (H)
    spatial_filter = H * np.exp(filter_exp)
    
    # Normalize the filter to ensure proper convolution
    spatial_filter /= np.sum(spatial_filter)
    
    return spatial_filter

# Generate synthetic bathymetry using a spatial filter
def generate_bathymetry_spatial_filter(grid_size, seafloor_age, sediment_thickness, params):
    """
    Generate synthetic bathymetry using a spatially varying filter based on von Kármán model.
    
    grid_size: tuple (nx, ny) specifying the size of the grid
    seafloor_age: 2D array of seafloor ages (used to calculate azimuth)
    sediment_thickness: 2D array of sediment thicknesses
    params: Dictionary containing base abyssal hill parameters
            e.g., {'H': H, 'kn': kn, 'ks': ks, 'D': D}
    """
    ny, nx = grid_size
    
    # Generate random noise field
    random_field = generate_random_field(grid_size)
    
    # Calculate azimuth from seafloor age gradient
    azimuth = calculate_azimuth_from_age(seafloor_age)
    
    # Initialize the output bathymetry array
    bathymetry = np.zeros(grid_size)
    
    def process_row(i):
        row_results = np.zeros(nx)
        print(i)

        for j in range(nx):
            # Extract local parameters
            H_local = params['H']
            kn_local = params['kn']
            ks_local = params['ks']
            azimuth_local = azimuth[i, j]

            # Modify the parameters based on sediment thickness
            H_local, kn_local, ks_local = modify_by_sediment(H_local, kn_local, ks_local, sediment_thickness[i, j])

            # Generate the local filter
            spatial_filter = generate_spatial_filter(H_local, kn_local, ks_local, azimuth_local)

            # Apply the filter to the random noise field at location (i, j)
            filtered_value = convolve(random_field, spatial_filter, mode='constant', cval=0.0)[i, j]

            # Store the filtered value in the row results
            row_results[j] = filtered_value

        return i, row_results
    
    '''
    # Loop over the grid to apply a spatial filter at each point
    for i in range(ny):
        print(i)
        for j in range(nx):
            # Extract local parameters
            H_local = params['H']
            kn_local = params['kn']
            ks_local = params['ks']
            azimuth_local = azimuth[i, j]
            
            # Modify the parameters based on sediment thickness
            H_local, kn_local, ks_local = modify_by_sediment(H_local, kn_local, ks_local, sediment_thickness[i, j])
            
            # Generate the local filter
            spatial_filter = generate_spatial_filter(H_local, kn_local, ks_local, azimuth_local)
            
            # Apply the filter to the random noise field at location (i, j)
            # Convolve the filter with the random field (centered at the current point)
            filtered_value = convolve(random_field, spatial_filter, mode='constant', cval=0.0)[i, j]
            
            # Store the filtered value in the bathymetry map
            bathymetry[i, j] = filtered_value
    
    return bathymetry
    '''
    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = [executor.submit(process_row, i) for i in range(ny)]

        # Collect results as they are completed
        for future in as_completed(futures):
            i, row_result = future.result()
            bathymetry[i, :] = row_result  # Insert the row into the bathymetry array
            
    return bathymetry



import os
os.cpu_count()



In [ ]:
# Example of input parameters (these would be derived from your grids)
params = {
    'H': 50,       # Base rms height in meters
    'kn': 0.05,   # Characteristic width in km^-1 (normal to ridge)
    'ks': 0.5,   # Characteristic length in km^-1 (parallel to ridge)
    'D': 2.2       # Fractal dimension
}

# Example grids (replace with real data)
nx, ny = 128, 128
#seafloor_age = np.random.rand(nx, ny) * 100  # Example seafloor age in Myr
sediment_thickness = np.random.rand(nx, ny) * 5  # Example sediment thickness in meters

seafloor_age,tmp = np.meshgrid(np.arange(0,256,1),np.arange(0,256,1))
#seafloor_age = seafloor_age+(sediment_thickness*2)
seafloor_age = seafloor_age+(tmp*2)


# Generate the synthetic bathymetry
synthetic_bathymetry = generate_bathymetry_spatial_filter((nx, ny), seafloor_age, sediment_thickness, params)

# Optionally, plot the bathymetry
import matplotlib.pyplot as plt

fig,ax = plt.subplots(ncols=2)
ax[0].imshow(seafloor_age)
ax[1].imshow(synthetic_bathymetry, cmap='terrain')
#plt.colorbar(label="Depth (m)")
#plt.title("Synthetic Abyssal Hill Bathymetry (Spatial Filter Method)")
plt.show()


In [ ]:
import xarray as xr

age_da = xr.open_dataarray('/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc')

print(age_da.shape)
age_da.plot()


In [ ]:
#nx = 360
#ny = 200
seafloor_age = age_da.data[400:800,200:1000]
ny,nx = seafloor_age.shape
sediment_thickness = np.random.rand(ny, nx) * 5  # Example sediment thickness in meters

# Generate the synthetic bathymetry
synthetic_bathymetry = generate_bathymetry_spatial_filter((ny, nx), seafloor_age, sediment_thickness, params)

# Optionally, plot the bathymetry
import matplotlib.pyplot as plt
plt.figure(figsize=(20,15))
plt.imshow(synthetic_bathymetry, cmap='terrain')
plt.colorbar(label="Depth (m)")
plt.title("Synthetic Abyssal Hill Bathymetry (Spatial Filter Method)")
plt.show()


In [ ]:
plt.imshow(seafloor_age)